# Setting up Colab


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
! pip install pysteps

# Imports


In [ ]:
import os
import joblib
import numpy as np
from pytz import timezone
from scipy.io import loadmat
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from pysteps.motion.lucaskanade import dense_lucaskanade
from pysteps.extrapolation.semilagrangian import extrapolate

Pysteps configuration file found at: /usr/local/lib/python3.10/dist-packages/pysteps/pystepsrc



# Utils

In [ ]:
def find_by_date(
    start_datetime, end_datetime,
    root_path, fn_pattern, fn_ext,
    timestep, error=2,
    upper=True, lower=False, verbose=False
):
    """
    start_datetime -> must be in the datetime format (inclusive of time)
    end_datetime   -> must be in the datetime format (inclusive of time)
    root_path      -> the path to the folder where all the files are stored
    fn_pattern     -> the general name of the file with the datetime pattern (strformat) embedded
    fn_ext         -> extension of the file, do NOT include the '.'
    timestep       -> in minutes,,, the time difference between two consecutive files
    error          -> in minutes,,, if the files are not exactly in a regular timestep, add the usual error (2 means +1, +2 min AND -1, -2 min)
    upper          -> convert the filename into upper case (some stupid thing with format)
    lower          -> convert the filename into lower case (added because just upper was weird)
    verbose        -> prints all the files not found

    **note** -> defaults are set up to work with satellite data from MOSDAC
    """

# error handling
    if upper and lower:
        raise ValueError("Cannot have both arguments, 'upper' and 'lower' be True!")

    if timestep <= 0 or isinstance(timestep, float):
        raise ValueError("Argument 'timestep' can only be a positive integer")

    if not (isinstance(start_datetime, datetime) and isinstance(end_datetime, datetime)):
        raise TypeError("Both 'start_datetime', 'end_datetime' arguments must be datetime objects")

    if not os.path.exists(root_path):
        raise IOError("Path provided in the 'root_path' argument does not exist")

    filenames = []
    timestamps = []

    cur_datetime = start_datetime

    while cur_datetime <= end_datetime + timedelta(minutes=error):
        fn = _find_matching_fn(
            cur_datetime, root_path, fn_pattern, fn_ext, error,
            upper, lower, verbose
        )

        filenames.append(fn)
        timestamps.append(cur_datetime)

    # iterating through the datetimes
        cur_datetime = cur_datetime + timedelta(minutes=timestep)

# this error exists because verbose might be false
    if all(filename is None for filename in filenames):
        raise IOError(f"No input data found in {root_path} with the given datetime range")

    return filenames, timestamps


def _find_matching_fn(
    cur_datetime, root_path, fn_pattern, fn_ext, error,
    upper, lower, verbose
):

# without error (timestep)
    fn = _gen_fn_path(
        cur_datetime,
        root_path, fn_pattern, fn_ext,
        upper, lower
    )

    if os.path.exists(fn):
        return fn

# with negative error
    for i in range(1, error+1):
        fn = _gen_fn_path(
            cur_datetime - timedelta(minutes=i),
            root_path, fn_pattern, fn_ext,
            upper, lower
        )
        if os.path.exists(fn):
            return fn

# with positive error
    for i in range(1, error+1):
        fn = _gen_fn_path(
            cur_datetime + timedelta(minutes=i),
            root_path, fn_pattern, fn_ext,
            upper, lower
        )
        if os.path.exists(fn):
            return fn

# nothing matched
    if verbose:
        error_date = datetime.strftime(cur_datetime, fn_pattern)
        if upper:
            error_date = error_date.upper()
        if lower:
            error_date = error_date.lower()

        print(f"file not found with datetime: {error_date}")

    return None


def _gen_fn_path(
    cur_datetime, root_path, fn_pattern, fn_ext,
    upper, lower
):
    fn = datetime.strftime(cur_datetime, fn_pattern)

    if upper:
        fn = fn.upper()
    if lower:
        fn = fn.lower()

    fn = fn + '.' + fn_ext
    fn = os.path.join(root_path, fn)

    return fn

In [ ]:
# decided by cursory empirical evidence
def norm(np_arr):
    return (np_arr - 80) / 400

def denorm(np_arr):
    return (np_arr * 400) + 80

In [ ]:
def data_year_generator(year, ts_yield=False, shuffle=True):
    load_params = {
        'start_datetime': datetime(year, 5, 1, 0, 0),
        'end_datetime': datetime(year, 9, 30, 23, 30),
        'root_path': f'/content/drive/MyDrive/ISRO/data/further_processed_May-Sep{year}',
        'fn_pattern': '%d%b%Y_%H%M_L2B_OLR',
        'fn_ext': 'npy',
        'timestep': 30,
        'upper': False
    }

    filenames, timestamps = find_by_date(**load_params)

# only the filenames and timesptamps that exist in the dataset
    fn_cleaned = []
    ts_cleaned = []
    for idx in range(len(filenames)):
        if filenames[idx] is not None:
            fn_cleaned.append(filenames[idx])
            ts_cleaned.append(timestamps[idx])

# shuffling the dataset
    if shuffle:
        shuffler = np.random.permutation(len(fn_cleaned))
        fn_cleaned = np.array(fn_cleaned)[shuffler]
        ts_cleaned = np.array(ts_cleaned)[shuffler]

# iterating through the entire dataset, batchwise
    for idx in range(0, len(fn_cleaned)):
        fn = fn_cleaned[idx]
        ts = ts_cleaned[idx]

        window = np.load(fn)

    # normalising
        window = np.where(np.isnan(window), 0, window)
        window = np.clip(window, a_min=80, a_max=None)      # to remove that weird plotting thing

    # split X and y
        X = window[:6]
        y = window[6:]

        result = (X, y)
        yield result + (ts,) if ts_yield else result

In [ ]:
flow_params = {
    'dense': True,
    'fd_method': 'blob',
    'interp_method': 'rbfinterp2d'
}

extrapolate_params = {
    'timesteps': 6,
    'interp_order': 3,
    'return_displacement': False,
}

# Building Motion Fields

do it only once, the training calls it from the disk to make sure we can train the model with data that doesnt entirely fit into memory

In [ ]:
save_path = '/content/drive/MyDrive/Nowcasting/final_b4_ppt/trad/'

for X, y, ts in data_year_generator(2022, True, False):
    fields_x = np.array([dense_lucaskanade(X[t:], **flow_params) for t in range(0, 6, 2)])
# have to experiment more as to what to keep as y_field
    # fields_y = dense_lucaskanade(np.concatenate((X[4:], y[:4]), axis=0), **flow_params)
    fields_y = dense_lucaskanade(y, **flow_params)

    np.savez(save_path + datetime.strftime(ts, '%d%b%Y_%H%M_regressor_data_motion_field'), x=fields_x, y=fields_y)

# Training Regressor

In [ ]:
neighbors_model = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)

load_params = {
    'start_datetime': datetime(2022, 5, 1, 0, 0),
    'end_datetime': datetime(2022, 9, 30, 23, 30),
    'root_path': '/content/drive/MyDrive/Nowcasting/final_b4_ppt/trad',
    'fn_pattern': '%d%b%Y_%H%M_regressor_data_motion_field',
    'fn_ext': 'npz',
    'timestep': 30,
    'upper': False
}
filenames, timestamps = find_by_date(**load_params)

fn_cleaned = []
ts_cleaned = []
for idx in range(len(filenames)):
    if filenames[idx] is not None:
        fn_cleaned.append(filenames[idx])
        ts_cleaned.append(timestamps[idx])

X = []
y = []
for fn in fn_cleaned:
    datum = np.load(fn)
    X.append(np.reshape(np.transpose(datum['x'], (2, 3, 0, 1)), (400*400, 3*2)))
    y.append(np.reshape(np.transpose(datum['y'], (1, 2, 0)), (400*400, 2)))

X = np.array(X)
X = np.reshape(X, (X.shape[0]*X.shape[1], X.shape[2]))

y = np.array(y)
y = np.reshape(y, (y.shape[0]*y.shape[1], y.shape[2]))

neighbors_model.fit(X, y)

dump_path = '/content/drive/MyDrive/Nowcasting/final_b4_ppt/trad/'
dump_path_final = dump_path + datetime.strftime(datetime.now(timezone('Asia/Kolkata')), '%d%b%Y_%H%M_knn_regressor.joblib')
joblib.dump(neighbors_model, dump_path_final)
print(dump_path_final)

/content/drive/MyDrive/Nowcasting/final_b4_ppt/trad/18Jul2023_1219_knn_regressor.joblib


# Testing

In [ ]:
loaded_model = joblib.load('/content/drive/MyDrive/Nowcasting/final_b4_ppt/trad/18Jul2023_1219_knn_regressor.joblib')

In [ ]:
for X, y, ts in data_year_generator(2021, True, True):
    fields_x = np.array([dense_lucaskanade(X[t:], **flow_params) for t in range(0, 6, 2)])
    fields_x_reshape = np.reshape(np.transpose(fields_x, (2, 3, 0, 1)), (400*400, 3*2))

    pred_field = loaded_model.predict(fields_x_reshape)
    pred_reshape_field = np.transpose(np.reshape(pred_field, (400, 400, 2)), (2, 0, 1))

    pred_olr_model = extrapolate(X[-1], pred_reshape_field, **extrapolate_params)

    print(ts, '\n')
    print("Model:")
    print("Average error across all 6 frames")
    print(np.nanmean(np.abs(y - pred_olr_model)))
    for i in range(6):
        print(f"Error for frame {i+1}")
        print(np.nanmean(np.abs(y[i] - pred_olr_model[i])))
    print('\n')

# using motion field from t-6 to t-1
    pred_olr_nomodel = extrapolate(X[-1], fields_x[0], **extrapolate_params)

    print('No Model:')
    print("Average error across all 6 frames")
    print(np.nanmean(np.abs(y - pred_olr_nomodel)))
    for i in range(6):
        print(f"Error for frame {i+1}")
        print(np.nanmean(np.abs(y[i] - pred_olr_nomodel[i])))
    print('\n')

2021-08-03 19:00:00 

Model:
Average error across all 6 frames
20.247765960679015
Error for frame 1
18.491522181283987
Error for frame 2
18.797964215273304
Error for frame 3
19.530184387563878
Error for frame 4
20.397595356739167
Error for frame 5
21.701483971748857
Error for frame 6
22.59782099613358


No Model:
Average error across all 6 frames
20.196198674973186
Error for frame 1
18.15991235605873
Error for frame 2
18.30878831756192
Error for frame 3
19.2014259257886
Error for frame 4
20.261053129801677
Error for frame 5
21.948525078215766
Error for frame 6
23.37096681592505


2021-06-14 01:00:00 

Model:
Average error across all 6 frames
17.541256420761798
Error for frame 1
9.076663167761517
Error for frame 2
13.690459173521564
Error for frame 3
16.85296405825199
Error for frame 4
19.490512099321567
Error for frame 5
22.08746700711976
Error for frame 6
24.152772282199393


No Model:
Average error across all 6 frames
17.22017470103408
Error for frame 1
8.719505435087605
Error for fr

KeyboardInterrupt: ignored